In [1]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pygame
import random
import sys

class PowerUp:
    def __init__(self, x, y, power_type):
        self.x = x
        self.y = y
        self.type = power_type
        self.width = 20
        self.height = 20
        self.color = {
            'speed': (255, 0, 0),    # Red
            'size': (0, 255, 0),      # Green
            'freeze': (0, 0, 255)     # Blue
        }[power_type]
        self.duration = 300  # 5 seconds at 60 FPS

    def draw(self, screen):
        """Draw power-up with a glowing effect"""
        glow_color = (self.color[0], self.color[1], self.color[2], 128)
        pygame.draw.circle(screen, glow_color, (self.x + self.width // 2, self.y + self.height // 2), self.width // 2 + 5)
        pygame.draw.circle(screen, self.color, (self.x + self.width // 2, self.y + self.height // 2), self.width // 2)

class PingPongGame:
    def __init__(self):
        pygame.init()
        
        # Screen Setup
        self.width = 1200
        self.height = 800
        self.screen = pygame.display.set_mode((self.width, self.height))
        pygame.display.set_caption("Advanced Ping Pong")
        
        # Colors
        self.WHITE = (255, 255, 255)
        self.BLACK = (0, 0, 0)
        
        # Game Objects
        self.paddle_width = 20
        self.paddle_height = 150
        self.ball_size = 20
        
        # Paddle Positions
        self.player1_x = 50
        self.player1_y = self.height // 2 - self.paddle_height // 2
        self.player2_x = self.width - 70
        self.player2_y = self.height // 2 - self.paddle_height // 2
        
        # Ball Physics
        self.ball_x = self.width // 2
        self.ball_y = self.height // 2
        self.ball_dx = 7 * random.choice((-1, 1))
        self.ball_dy = 7 * random.choice((-1, 1))
        
        # Game State
        self.player1_score = 0
        self.player2_score = 0
        self.clock = pygame.time.Clock()
        self.font = pygame.font.Font(None, 50)
        
        # Power-up System
        self.power_ups = []
        self.active_power_ups = {'player1': None, 'player2': None}
        self.power_up_spawn_timer = 0
        
        # Movement Speed
        self.player1_speed = 8
        self.player2_speed = 8
        
        # Difficulty Progression
        self.difficulty_multiplier = 1.0

    def spawn_power_up(self):
        """Randomly spawn power-ups on the screen"""
        power_types = ['speed', 'size', 'freeze']
        x = random.randint(200, self.width - 200)
        y = random.randint(100, self.height - 100)
        return PowerUp(x, y, random.choice(power_types))
    
    def apply_power_up(self, player, power_up):
        """Apply power-up effects to players"""
        if player == 'player1':
            if power_up.type == 'speed':
                self.player1_speed *= 1.5
            elif power_up.type == 'size':
                self.paddle_height *= 1.5
            elif power_up.type == 'freeze':
                self.player2_speed = 2
        else:
            if power_up.type == 'speed':
                self.player2_speed *= 1.5
            elif power_up.type == 'size':
                self.paddle_height *= 1.5
            elif power_up.type == 'freeze':
                self.player1_speed = 2
    
    def reset_power_ups(self):
        """Reset power-up effects"""
        self.player1_speed = 8
        self.player2_speed = 8
        self.paddle_height = 150
        
    def handle_events(self):
        """Handle keyboard events for player movement"""
        keys = pygame.key.get_pressed()
        
        # Player 1 Controls (W/S)
        if keys[pygame.K_w] and self.player1_y > 0:
            self.player1_y -= self.player1_speed
        if keys[pygame.K_s] and self.player1_y < self.height - self.paddle_height:
            self.player1_y += self.player1_speed
        
        # Player 2 Controls (UP/DOWN Arrow)
        if keys[pygame.K_UP] and self.player2_y > 0:
            self.player2_y -= self.player2_speed
        if keys[pygame.K_DOWN] and self.player2_y < self.height - self.paddle_height:
            self.player2_y += self.player2_speed
    
    def update_ball(self):
        """Update ball movement and handle collisions"""
        self.ball_x += self.ball_dx
        self.ball_y += self.ball_dy
        
        # Wall collisions
        if self.ball_y <= 0 or self.ball_y >= self.height - self.ball_size:
            self.ball_dy *= -1
        
        # Paddle collisions
        if (self.ball_x <= self.player1_x + self.paddle_width and 
            self.player1_y <= self.ball_y <= self.player1_y + self.paddle_height):
            self.ball_dx *= -1.1
            
        if (self.ball_x + self.ball_size >= self.player2_x and 
            self.player2_y <= self.ball_y <= self.player2_y + self.paddle_height):
            self.ball_dx *= -1.1
        
        # Scoring
        if self.ball_x < 0:
            self.player2_score += 1
            self.reset_ball()
        
        if self.ball_x > self.width:
            self.player1_score += 1
            self.reset_ball()
    
    def reset_ball(self):
        """Reset ball position and speed"""
        self.ball_x = self.width // 2
        self.ball_y = self.height // 2
        self.ball_dx = 7 * random.choice((-1, 1))
        self.ball_dy = 7 * random.choice((-1, 1))
        self.reset_power_ups()

    def draw_gradient_background(self):
        """Draw a gradient background"""
        for i in range(self.height):
            r = int(0 * (i / self.height))
            g = int(50 * (i / self.height))
            b = int(200 * (i / self.height))
            pygame.draw.line(self.screen, (r, g, b), (0, i), (self.width, i))
    
    def draw_game(self):
        """Render game objects"""
        self.draw_gradient_background()
        
        # Draw Paddles with rounded edges
        pygame.draw.rect(self.screen, self.WHITE, 
                         (self.player1_x, self.player1_y, 
                          self.paddle_width, self.paddle_height), border_radius=10)
        pygame.draw.rect(self.screen, self.WHITE, 
                         (self.player2_x, self.player2_y, 
                          self.paddle_width, self.paddle_height), border_radius=10)
        
        # Draw Ball with glow effect
        pygame.draw.circle(self.screen, (255, 255, 255), (self.ball_x + self.ball_size // 2, self.ball_y + self.ball_size // 2), self.ball_size // 2)
        
        # Draw Scores with glow effect
        score1 = self.font.render(f"Player 1: {self.player1_score}", True, (255, 215, 0))  # Gold color
        score2 = self.font.render(f"Player 2: {self.player2_score}", True, (255, 215, 0))  # Gold color
        self.screen.blit(score1, (50, 50))
        self.screen.blit(score2, (self.width - 250, 50))
        
        # Draw Power-ups
        for power_up in self.power_ups:
            power_up.draw(self.screen)
        
        pygame.display.flip()
    
    def run(self):
        """Main game loop"""
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
            
            self.handle_events()
            self.update_ball()
            
            # Power-up spawning
            self.power_up_spawn_timer += 1
            if self.power_up_spawn_timer > 600:  # Spawn every 10 seconds
                self.power_ups.append(self.spawn_power_up())
                self.power_up_spawn_timer = 0
            
            self.draw_game()
            self.clock.tick(60)  # 60 FPS
        
        pygame.quit()
        sys.exit()

# Run the game
if __name__ == "__main__":
    game = PingPongGame()
    game.run()
